In [ ]:
%pip install kobert_transformers

In [ ]:
# 구글 코랩에서 실행 권장(gpu 권장)
import pandas as pd
import torch
from transformers import BertModel
from kobert_transformers import get_tokenizer, get_kobert_model
from transformers import BertTokenizer
import numpy as np
from io import StringIO
import multiprocessing as mp
from google.colab import files

model = get_kobert_model()
tokenizer = get_tokenizer()

def get_document_embedding(sentences):
    inputs = tokenizer(
        sentences,
        return_tensors='pt',
        padding=True,
        truncation=True
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get the sentence embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings for the [CLS] token (shape: [batch_size, hidden_size])
    cls_embeddings = outputs.last_hidden_state[:, 0, :]

    # Pool the sentence embeddings to create a single document embedding by averaging
    document_embedding = torch.mean(cls_embeddings, dim=0)

    return document_embedding.cpu().numpy()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
july_2024 = pd.read_csv("/content/drive/Othercomputers/갤럭시북4pro/workspace/july_2024.csv", index_col=0)
samples = list(july_2024['clear_sentence'])

In [ ]:
embedded_list = []
i = 0
j = 0
for doc in samples:
  try:
    embedded_list.append(get_document_embedding(doc))
  except Exception as e:
    i += 1
    embedded_list.append('e')
    print(f'에러발생 {i}회, index = {samples.index(doc)}')
  if samples.index(doc) % 1000 == 0:
    print(f'{j*1000}개 완료')
    j += 1

pd.DataFrame({'vector':embedded_list}).to_csv('/content/drive/Othercomputers/갤럭시북4pro/workspace/embedded_data1_july_2024.csv')


print("Complete")